In [42]:
import pickle
import numpy as np 
import pandas as pd 
import math

import sys
sys.path.append('../utils')
from treasury_base import *
from scipy.stats import t
from sklearn.metrics import mean_squared_error as mse

In [43]:
cross_val_window = 20
with open(f"pickle/dfs_{cross_val_window}_steps_ahead.pkl", "rb") as file:
    loaded_dataframes = pickle.load(file)

# Extract individual DataFrames
naive_df = loaded_dataframes["naive_df"]
kan_df = loaded_dataframes["kan_df"]
truth_df = loaded_dataframes["truth_df"]

# Use the DataFrames
kan_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,4.934075,4.992681,5.001265,4.698945,4.443489,4.326618,4.310841,4.243105,4.278586,4.402402,4.541232,4.466565
1,5.028719,5.042461,5.041642,4.733073,4.412884,4.276511,4.251580,4.167601,4.213121,4.343022,4.506578,4.393338
2,5.145692,5.129354,5.135878,4.791160,4.417996,4.233332,4.243721,4.102390,4.166791,4.273433,4.443533,4.363269
3,4.996382,5.177554,5.123763,4.756830,4.597951,4.249633,4.236229,4.088197,4.165624,4.336492,4.463708,4.446730
4,5.044475,5.183874,5.144227,4.766924,4.475250,4.113892,4.134752,4.021145,4.097930,4.292274,4.344559,4.391781


In [58]:
cols = truth_df.columns
mode = 'two-sided'
cross_val_list = [1, 5, 10, 20]

result = dict()
for i in range(12):
    result[i] = list()

for cross_val_window in cross_val_list:
    with open(f"pickle/dfs_{cross_val_window}_steps_ahead.pkl", "rb") as file:
        loaded_dataframes = pickle.load(file)

    # Extract individual DataFrames
    naive_df = loaded_dataframes["naive_df"]
    kan_df = loaded_dataframes["kan_df"]
    truth_df = loaded_dataframes["truth_df"]

    for i in range(12):
        y1 = kan_df.iloc[:, i].values
        y2 = naive_df.iloc[:, i].values
        y = truth_df.iloc[:, i].values

        # print(f'{cols[i]}: {dm_test(y, y1, y2, h=1, mode=mode)}')
        # print(f'{cols[i]}, h={cross_val_window}: {mse(y1, y, squared=False):.2f}, {mse(y2, y, squared=False):.2f}')
        if mse(y1, y) < mse(y2, y):
            result[i].append(True)
        else:
            result[i].append(False)

out = pd.DataFrame(result)
out.columns = cols
out.index = cross_val_list
out


,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
1,False,False,False,False,False,False,False,False,False,False,False,False
5,True,False,False,False,False,False,False,False,False,False,False,False
10,True,False,False,False,False,False,False,False,True,False,True,True
20,False,False,False,False,False,True,True,True,True,True,True,True


In [49]:
print(mse(kan_df.values, truth_df.values, squared=False))
print(mse(naive_df.values, truth_df.values, squared=False))

0.2126718906519023
0.12692289726785774
